<a href="https://colab.research.google.com/github/Justy-11/Analysis-of-Transfer-Learning-for-Image-Classification-on-Sparse-Data/blob/main/IRP_Preprocessing_skin_disease_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import shutil
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
# Move the API key to the appropriate directory
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# download skin disease dataset from kaggle API command
!kaggle datasets download -d riyaelizashaju/skin-disease-classification-image-dataset

 94% 159M/169M [00:01<00:00, 113MB/s] 
100% 169M/169M [00:01<00:00, 96.9MB/s]


In [ ]:
# unzip dataset
!unzip skin-disease-classification-image-dataset.zip

Archive:  skin-disease-classification-image-dataset.zip
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024468.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024470.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024707.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024763.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024771.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024800.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024913.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024948.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0025780.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0025953.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0025957.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0025992.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0026149.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0026171.jpg  
  infl

In [ ]:
# move dataset to drive
!mv /content/Split_smol /content/drive/MyDrive/datasets

In [ ]:
import os
import shutil

# Define paths
base_path = "/content/drive/MyDrive/datasets/Split_smol"
train_path = os.path.join(base_path, "train")
val_path = os.path.join(base_path, "val")

# List of class names
class_names = ["Actinic keratosis", "Atopic Dermatitis", "Benign keratosis",
               "Dermatofibroma", "Melanocytic nevus", "Melanoma",
               "Squamous cell carcinoma", "Tinea Ringworm Candidiasis",
               "Vascular lesion"]

# Move images from val to train
for class_name in class_names:
    train_class_path = os.path.join(train_path, class_name)
    val_class_path = os.path.join(val_path, class_name)

    # Move images
    for filename in os.listdir(val_class_path):
        src = os.path.join(val_class_path, filename)
        dst = os.path.join(train_class_path, filename)

        # Handle filename conflicts by adding a suffix
        counter = 1
        while os.path.exists(dst):
            base, extension = os.path.splitext(filename)
            new_filename = f"{base}_{counter}{extension}"
            dst = os.path.join(train_class_path, new_filename)
            counter += 1

        shutil.move(src, dst)

print("Images moved successfully.")

Images moved successfully.


In [ ]:
new_folders = ["Actinic keratosis", "Atopic Dermatitis", "Benign keratosis",
               "Dermatofibroma", "Melanocytic nevus", "Melanoma",
               "Squamous cell carcinoma", "Tinea Ringworm Candidiasis",
               "Vascular lesion"]

# count the images
for folder in new_folders:
    folder_path = os.path.join("/content/drive/MyDrive/datasets/Split_smol/train", folder)

    # Count the number of images in the folder
    num_images = len(os.listdir(folder_path))
    print(f"{folder}: {num_images} images")

Actinic keratosis: 100 images
Atopic Dermatitis: 102 images
Benign keratosis: 100 images
Dermatofibroma: 100 images
Melanocytic nevus: 100 images
Melanoma: 100 images
Squamous cell carcinoma: 100 images
Tinea Ringworm Candidiasis: 76 images
Vascular lesion: 100 images


In [ ]:
# change image names
# Define the folders inside the selected images folder
selected_folders = ["Actinic keratosis", "Atopic Dermatitis", "Benign keratosis",
               "Dermatofibroma", "Melanocytic nevus", "Melanoma",
               "Squamous cell carcinoma", "Tinea Ringworm Candidiasis",
               "Vascular lesion"]

selected_images_folder = '/content/drive/MyDrive/datasets/Split_smol/train'

# Mapping of folder names to abbreviations
folder_name_to_abbreviation = {
    "Actinic keratosis": "Ak",
    "Atopic Dermatitis": "AD",
    "Benign keratosis": "Bk",
    "Dermatofibroma": "D",
    "Melanocytic nevus": "Mn",
    "Melanoma": "M",
    "Squamous cell carcinoma": "Scc",
    "Tinea Ringworm Candidiasis": "TRC",
    "Vascular lesion": "Vl"
}

# Rename and count the images
for folder in selected_folders:
    folder_path = os.path.join(selected_images_folder, folder)
    all_images = os.listdir(folder_path)

    # Rename the images using the corresponding abbreviation
    for idx, image in enumerate(all_images, start=1):
        abbreviation = folder_name_to_abbreviation.get(folder, folder[:2].upper())  # Use first two characters as abbreviation if not found
        new_filename = f"{abbreviation}_{idx}.JPG"
        source_path = os.path.join(folder_path, image)
        target_path = os.path.join(folder_path, new_filename)
        os.rename(source_path, target_path)

    # Count the number of images in the folder
    num_images = len(os.listdir(folder_path))
    print(f"{folder}: {num_images} images")

Actinic keratosis: 100 images
Atopic Dermatitis: 102 images
Benign keratosis: 100 images
Dermatofibroma: 100 images
Melanocytic nevus: 100 images
Melanoma: 100 images
Squamous cell carcinoma: 100 images
Tinea Ringworm Candidiasis: 76 images
Vascular lesion: 100 images


In [ ]:
base_dir = "/content/drive/MyDrive/datasets/Split_smol/train"
img_size = 224

# 5 class labels
class_labels = {
    "Actinic keratosis": 0,
    "Atopic Dermatitis": 1,
    "Benign keratosis": 2,
    "Dermatofibroma": 3,
    "Melanocytic nevus": 4
}

traindata = []  # List to store [img_array, label] pairs

# Loop through the selected classes
for class_folder in class_labels:
    class_folder_path = os.path.join(base_dir, class_folder)

    if os.path.isdir(class_folder_path):
        img_files = [f for f in os.listdir(class_folder_path) if f.endswith(('.JPG'))]

        # Loop through all images in the class folder
        for image_file in img_files:
            image_path = os.path.join(class_folder_path, image_file)

            # Read the image
            image = cv2.imread(image_path)

            # Check if the image is in RGB format
            if image.shape[2] == 3:
                new_img_array = cv2.resize(image, (img_size, img_size))
            else:
                # Convert BGR to RGB
                back_to_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                new_img_array = cv2.resize(back_to_rgb, (img_size, img_size))

            # Assign label based on class_labels dictionary
            label = class_labels[class_folder]

            # Append the image array and label to the training data
            traindata.append([new_img_array, label])

# Shuffle the training data
np.random.shuffle(traindata)

In [ ]:
len(traindata)

502

In [ ]:
traindata[0][0].shape

(224, 224, 3)

In [ ]:
traindata[7][1]

2

In [ ]:
X = []
y = []
img_size = 224

for features, labels in traindata:
    X.append(features)
    y.append(labels)

# reshape
X = np.array(X).reshape(-1, img_size, img_size, 3)
y = np.array(y)

In [ ]:
X.shape, y.shape

((502, 224, 224, 3), (502,))

In [ ]:
# create new diectory to save models
# Specify the path of the existing folder
existing_folder_path = '/content/drive/MyDrive/datasets/Split_smol'

# Specify the name of the new directory
new_directory_name = 'models'

# Create the full path for the new directory
new_directory_path = os.path.join(existing_folder_path, new_directory_name)

# Check if the directory already exists, if not, create it
if not os.path.exists(new_directory_path):
    os.makedirs(new_directory_path)
    print(f"Directory '{new_directory_name}' created successfully.")
else:
    print(f"Directory '{new_directory_name}' already exists.")

Directory 'models' created successfully.


In [ ]:
# create new diectory to save samples
# Specify the path of the existing folder
existing_folder_path = '/content/drive/MyDrive/datasets/Split_smol'

# Specify the name of the new directory
new_directory_name = 'samples'

# Create the full path for the new directory
new_directory_path = os.path.join(existing_folder_path, new_directory_name)

# Check if the directory already exists, if not, create it
if not os.path.exists(new_directory_path):
    os.makedirs(new_directory_path)
    print(f"Directory '{new_directory_name}' created successfully.")
else:
    print(f"Directory '{new_directory_name}' already exists.")

Directory 'samples' created successfully.


In [ ]:
def create_folder(existing_folder_path, new_directory_name):
  # Create the full path for the new directory
  new_directory_path = os.path.join(existing_folder_path, new_directory_name)

  # Check if the directory already exists, if not, create it
  if not os.path.exists(new_directory_path):
    os.makedirs(new_directory_path)
    print(f"Directory '{new_directory_name}' created successfully.")
  else:
    print(f"Directory '{new_directory_name}' already exists.")

In [ ]:
create_folder('/content/drive/MyDrive/datasets/Split_smol/samples', 'sample1')
create_folder('/content/drive/MyDrive/datasets/Split_smol/samples', 'sample2')
create_folder('/content/drive/MyDrive/datasets/Split_smol/samples', 'sample3')

Directory 'sample1' created successfully.
Directory 'sample2' created successfully.
Directory 'sample3' created successfully.


In [ ]:
# first separate the test set - keep test common for all sample sizes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(426, 224, 224, 3) (426,) (76, 224, 224, 3) (76,)


In [ ]:
create_folder('/content/drive/MyDrive/datasets/Split_smol/samples', 'test_set')

Directory 'test_set' created successfully.


In [ ]:
# save test set
save_dir = '/content/drive/MyDrive/datasets/Split_smol/samples/test_set/'

np.save(save_dir + 'X_test.npy', X_test)
np.save(save_dir + 'y_test.npy', y_test)

In [ ]:
# sample creation - sample 2, 3
X_sample_2, X_temp, y_sample_2, y_temp = train_test_split(X_train, y_train, test_size=0.5, random_state=42, stratify=y_train)
X_sample_3, X_temp, y_sample_3, y_temp = train_test_split(X_train, y_train, test_size=0.75, random_state=42, stratify=y_train)

In [ ]:
print(X_sample_2.shape, y_sample_2.shape, X_sample_3.shape, y_sample_3.shape)

(213, 224, 224, 3) (213,) (106, 224, 224, 3) (106,)


In [ ]:
def create_splits_and_save(X, y, save_dir):
  X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=75/426, random_state=42, stratify=y)
  # save
  np.save(save_dir + 'X_train.npy', X_train)
  np.save(save_dir + 'X_val.npy', X_val)
  np.save(save_dir + 'y_train.npy', y_train)
  np.save(save_dir + 'y_val.npy', y_val)

  print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

In [ ]:
# sample splits
create_splits_and_save(X_train, y_train, '/content/drive/MyDrive/datasets/Split_smol/samples/sample1/')
create_splits_and_save(X_sample_2, y_sample_2, '/content/drive/MyDrive/datasets/Split_smol/samples/sample2/')
create_splits_and_save(X_sample_3, y_sample_3, '/content/drive/MyDrive/datasets/Split_smol/samples/sample3/')

(351, 224, 224, 3) (351,) (75, 224, 224, 3) (75,)
(175, 224, 224, 3) (175,) (38, 224, 224, 3) (38,)
(87, 224, 224, 3) (87,) (19, 224, 224, 3) (19,)


In [ ]:
np.sum(y_sample_2==0)

42

In [ ]:
# load train data
save_dir_sample3 = '/content/drive/MyDrive/datasets/Split_smol/samples/sample3/'
save_dir_sample2 = '/content/drive/MyDrive/datasets/Split_smol/samples/sample2/'
save_dir_sample1 = '/content/drive/MyDrive/datasets/Split_smol/samples/sample1/'

X_train_3 = np.load(save_dir_sample3 + 'X_train.npy')
y_train_3 = np.load(save_dir_sample3 + 'y_train.npy')

X_train_2 = np.load(save_dir_sample2 + 'X_train.npy')
y_train_2 = np.load(save_dir_sample2 + 'y_train.npy')

X_train_1 = np.load(save_dir_sample1 + 'X_train.npy')
y_train_1 = np.load(save_dir_sample1 + 'y_train.npy')

In [ ]:
# Concatenate data and labels
X_train = np.concatenate((X_train_1, X_train_2, X_train_3), axis=0)
y_train = np.concatenate((y_train_1, y_train_2, y_train_3), axis=0)

In [ ]:
# Get the number of samples
num_samples = X_train.shape[0]

# Generate a random permutation of indices
permutation = np.random.permutation(num_samples)

# Apply the same permutation to both X_train and y_train
X_train_shuffled = X_train[permutation]
y_train_shuffled = y_train[permutation]

In [ ]:
def custom_data_augmentation(images, labels):
    augmented_images = []
    augmented_labels = []

    unique_labels = np.unique(labels)

    for label in unique_labels:
        # Find indices of images belonging to the current class
        indices = np.where(labels == label)[0]

        for index in indices:
            # Apply a random augmentation to every image in the class
            random_choice = np.random.choice([0, 1, 2, 3, 4], size=1)[0]

            if random_choice == 0:
                # Horizontal flip
                augmented_images.append(cv2.flip(images[index], 1))
                augmented_labels.append(label)
            elif random_choice == 1:
                # Vertical flip
                augmented_images.append(cv2.flip(images[index], 0))
                augmented_labels.append(label)
            elif random_choice == 2:
                # 90-degree rotation
                augmented_images.append(np.rot90(images[index], k=1, axes=(0, 1)))
                augmented_labels.append(label)
            elif random_choice == 3:
                # 180-degree rotation
                augmented_images.append(np.rot90(images[index], k=2, axes=(0, 1)))
                augmented_labels.append(label)
            else:
                # 270-degree rotation
                augmented_images.append(np.rot90(images[index], k=3, axes=(0, 1)))
                augmented_labels.append(label)

    return np.array(augmented_images), np.array(augmented_labels)

In [ ]:
augmented_X_train, augmented_y_train = custom_data_augmentation(X_train_shuffled, y_train_shuffled)

In [ ]:
X_train_shuffled.shape, y_train_shuffled.shape, augmented_X_train.shape, augmented_y_train.shape

((613, 224, 224, 3), (613,), (613, 224, 224, 3), (613,))

In [ ]:
y_train_shuffled[11], augmented_y_train[500]

(1, 4)

In [ ]:
np.sum(augmented_y_train==4)

122

In [ ]:
del num_samples
del permutation

In [ ]:
# Get the number of samples
num_samples = augmented_X_train.shape[0]

# Generate a random permutation of indices
permutation = np.random.permutation(num_samples)

# Apply the same permutation to both X_train and y_train
X_train_augmented = augmented_X_train[permutation]
y_train_augmented = augmented_y_train[permutation]

In [ ]:
np.sum(y_train_augmented==4)

122

In [ ]:
X_train_augmented.shape, X_train_1.shape, X_train_2.shape, X_train_3.shape

((613, 224, 224, 3), (351, 224, 224, 3), (175, 224, 224, 3), (87, 224, 224, 3))

In [ ]:
def create_folder(existing_folder_path, new_directory_name):
  # Create the full path for the new directory
  new_directory_path = os.path.join(existing_folder_path, new_directory_name)

  # Check if the directory already exists, if not, create it
  if not os.path.exists(new_directory_path):
    os.makedirs(new_directory_path)
    print(f"Directory '{new_directory_name}' created successfully.")
  else:
    print(f"Directory '{new_directory_name}' already exists.")

In [ ]:
create_folder('/content/drive/MyDrive/datasets/Split_smol', 'DA_2')

Directory 'DA_2' created successfully.


In [ ]:
create_folder('/content/drive/MyDrive/datasets/Split_smol/DA_2', 'sample1')
create_folder('/content/drive/MyDrive/datasets/Split_smol/DA_2', 'sample2')
create_folder('/content/drive/MyDrive/datasets/Split_smol/DA_2', 'sample3')

Directory 'sample1' created successfully.
Directory 'sample2' created successfully.
Directory 'sample3' created successfully.


In [ ]:
X_train_augmented_1, X_temp, y_train_augmented_1, y_temp = train_test_split(X_train_augmented, y_train_augmented, test_size=262/613, random_state=42, stratify=y_train_augmented)

In [ ]:
X_train_augmented_1.shape, np.sum(y_train_augmented_1==0)

((351, 224, 224, 3), 70)

In [ ]:
X_train_augmented_2, X_temp, y_train_augmented_2, y_temp = train_test_split(X_train_augmented, y_train_augmented, test_size=438/613, random_state=42, stratify=y_train_augmented)

In [ ]:
X_train_augmented_2.shape, np.sum(y_train_augmented_2==4)

((175, 224, 224, 3), 35)

In [ ]:
X_train_augmented_3, X_temp, y_train_augmented_3, y_temp = train_test_split(X_train_augmented, y_train_augmented, test_size=526/613, random_state=42, stratify=y_train_augmented)

In [ ]:
X_train_augmented_3.shape, np.sum(y_train_augmented_3==0)

((87, 224, 224, 3), 17)

In [ ]:
# Concatenate the original and augmented datasets
X_combined_1 = np.concatenate([X_train_1, X_train_augmented_1])
y_combined_1 = np.concatenate([y_train_1, y_train_augmented_1])

# Create shuffled indices
shuffled_indices = np.random.permutation(len(X_combined_1))

# Shuffle both arrays using the same indices
X_final_1 = X_combined_1[shuffled_indices]
y_final_1 = y_combined_1[shuffled_indices]

# Concatenate the original and augmented datasets for subset 2
X_combined_2 = np.concatenate([X_train_2, X_train_augmented_2])
y_combined_2 = np.concatenate([y_train_2, y_train_augmented_2])

# Create shuffled indices
shuffled_indices_2 = np.random.permutation(len(X_combined_2))

# Shuffle both arrays using the same indices
X_final_2 = X_combined_2[shuffled_indices_2]
y_final_2 = y_combined_2[shuffled_indices_2]

# Concatenate the original and augmented datasets for subset 3
X_combined_3 = np.concatenate([X_train_3, X_train_augmented_3])
y_combined_3 = np.concatenate([y_train_3, y_train_augmented_3])

# Create shuffled indices
shuffled_indices_3 = np.random.permutation(len(X_combined_3))

# Shuffle both arrays using the same indices
X_final_3 = X_combined_3[shuffled_indices_3]
y_final_3 = y_combined_3[shuffled_indices_3]

In [ ]:
print("Shapes for Subset 1:")
print("X_final_1:", X_final_1.shape)
print("y_final_1:", y_final_1.shape)

print("\nShapes for Subset 2:")
print("X_final_2:", X_final_2.shape)
print("y_final_2:", y_final_2.shape)

print("\nShapes for Subset 3:")
print("X_final_3:", X_final_3.shape)
print("y_final_3:", y_final_3.shape)

Shapes for Subset 1:
X_final_1: (702, 224, 224, 3)
y_final_1: (702,)

Shapes for Subset 2:
X_final_2: (350, 224, 224, 3)
y_final_2: (350,)

Shapes for Subset 3:
X_final_3: (174, 224, 224, 3)
y_final_3: (174,)


In [ ]:
# Specify the directory paths
save_dir_sample1 = '/content/drive/MyDrive/datasets/Split_smol/DA_2/sample1'
save_dir_sample2 = '/content/drive/MyDrive/datasets/Split_smol/DA_2/sample2'
save_dir_sample3 = '/content/drive/MyDrive/datasets/Split_smol/DA_2/sample3'

# Save final arrays for Subset 1
np.save(os.path.join(save_dir_sample1, 'X_train.npy'), X_final_1)
np.save(os.path.join(save_dir_sample1, 'y_train.npy'), y_final_1)

# Save final arrays for Subset 2
np.save(os.path.join(save_dir_sample2, 'X_train.npy'), X_final_2)
np.save(os.path.join(save_dir_sample2, 'y_train.npy'), y_final_2)

# Save final arrays for Subset 3
np.save(os.path.join(save_dir_sample3, 'X_train.npy'), X_final_3)
np.save(os.path.join(save_dir_sample3, 'y_train.npy'), y_final_3)